In [1]:
import pandas as pd
from datetime import datetime

In [2]:
racers_df = pd.read_csv('data/FnGRacers.csv')
races_df = pd.read_csv('data/FnGRaces.csv')
results_df = pd.read_csv('data/FnGResults.csv')

In [3]:
# Extracting the year from race dates
races_df['year'] = pd.to_datetime(races_df['race_date']).dt.year
combined_df = results_df.merge(races_df[['race_id', 'year']], on='race_id')
combined_df = combined_df.merge(racers_df, how='left', on=['racer_id'])
# Remove whitespaces from discipline
combined_df['discipline'] = combined_df['discipline'].str.strip()

In [4]:
# Function to calculate aggregated and last 3 years race result percentiles for a given dataset
def calculate_aggregated_percentiles(df):
    # Calculating percentile ranks for each race
    # Lower times are better, so we use ascending=True
    df['_percentile'] = df.groupby('race_id')['_time'].rank(pct=True, ascending=True)
    
    # Calculating median percentile rank for each racer per year
    median_percentiles = df.groupby(['racer_id', 'year'])['_percentile'].median().reset_index()
    
    # Creating a dictionary to hold pre-season and post-season rankings for each year
    rankings = {}
    
    # Iterating over each year present in the dataset
    for year in sorted(median_percentiles['year'].unique()):
        # Getting the median percentiles for the current year
        current_year_rankings = median_percentiles[median_percentiles['year'] == year]
    
        # Sorting by median percentile (better performance has lower percentile)
        current_year_rankings = current_year_rankings.sort_values('_percentile')
    
        # Assigning post-season rankings based on sorted order
        current_year_rankings['post_season_rank'] = range(1, len(current_year_rankings) + 1)
    
        # Assigning pre-season rankings
        if year - 1 in rankings:
            # If the previous year exists, use its post-season rankings
            previous_year_rankings = rankings[year - 1][['racer_id', 'post_season_rank']]
            current_year_rankings = current_year_rankings.merge(previous_year_rankings, on='racer_id', how='left', suffixes=('', '_prev'))
            current_year_rankings['pre_season_rank'] = current_year_rankings['post_season_rank_prev'].fillna(max(current_year_rankings['post_season_rank']) + 1)
            current_year_rankings.drop(columns=['post_season_rank_prev'], inplace=True)
        else:
            # If no previous year data, assign a default pre-season rank
            current_year_rankings['pre_season_rank'] = max(current_year_rankings['post_season_rank']) + 1
    
        # Adding to the rankings dictionary
        rankings[year] = current_year_rankings
    
    # Combining all years' rankings into a single DataFrame
    all_years_rankings = pd.concat(rankings.values())
    
    # Calculating all-time percentile ranking for each racer: TODO, update this!
    all_time_percentiles = df.groupby('racer_id')['_percentile'].mean().reset_index()
    all_time_percentiles.rename(columns={'_percentile': 'all_time_avg_percentile'}, inplace=True)

    # Getting the last 3 years
    last_3_years = df['year'].unique()[-3:]
    columns = []

    # Initializing a DataFrame to hold last 3 years' percentiles
    last_3_years_percentiles = pd.DataFrame()

    # Iterating over each of the last 3 years
    for year in last_3_years:
        # Calculating median percentile for each racer for the given year
        year_percentiles = df[df['year'] == year].groupby('racer_id')['_percentile'].median().reset_index()
        year_percentiles.rename(columns={'_percentile': f'median_percentile_{year}'}, inplace=True)
        columns.append(f'median_percentile_{year}')

        # Merging with the main DataFrame
        if last_3_years_percentiles.empty:
            last_3_years_percentiles = year_percentiles
        else:
            last_3_years_percentiles = last_3_years_percentiles.merge(year_percentiles, on='racer_id', how='outer')

    # Merging all-time percentiles with last 3 years' percentiles
    aggregated_df = all_time_percentiles.merge(last_3_years_percentiles, on='racer_id', how='left')

    # Finally, create a ranking from the last 3 years and avg data:    
    aggregated_df['rank'] = aggregated_df.apply(
        lambda x: x[columns[-1]] if pd.notnull(x[columns[-1]]) else x[columns[-2]] if pd.notnull(x[columns[-2]]) else x[columns[-3]] if pd.notnull(x[columns[-3]]) else x['all_time_avg_percentile'],
        axis=1
    )

    return aggregated_df, all_years_rankings


In [5]:
ski_df = combined_df[combined_df['discipline'] == 'SKI']
snbd_df = combined_df[combined_df['discipline'] == 'SNBD']

In [6]:
# Calculating for SKI and SNBD
ski_aggregated_df, ski_all_time = calculate_aggregated_percentiles(ski_df)
snbd_aggregated_df, snbd_all_time = calculate_aggregated_percentiles(snbd_df)

/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_47623/1779360528.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_percentile'] = df.groupby('race_id')['_time'].rank(pct=True, ascending=True)
/var/folders/xl/qrck89yn5kq2bhd8vf0657700000gn/T/ipykernel_47623/1779360528.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['_percentile'] = df.groupby('race_id')['_time'].rank(pct=True, ascending=True)


# Finally, create the ranking

In [7]:
# Current sign ups
eligible_pop = pd.read_csv('data/FnG2024Racers.csv')
eligible_pop["racer_id"] = eligible_pop["Last Name"] + ", " + eligible_pop["First Name"]
eligible_pop.head(2)

,memberID,First Name,Last Name,Age,Gender,Program,racer_id
0,P777,Jason,Park,53.0,M,SKI,"Park, Jason"
1,M111,Rachel,Megitt,43.0,F,SKI,"Megitt, Rachel"


In [8]:
racer_columns = ['racer_id', 'Age', 'Gender']

In [9]:
ski_list = eligible_pop[eligible_pop.Program == 'SKI'][racer_columns].merge(ski_aggregated_df, how="left", on="racer_id")

In [10]:
snbd_list = eligible_pop[eligible_pop.Program == 'SNBD'][racer_columns].merge(ski_aggregated_df, how="left", on="racer_id")

In [12]:
ski_list.sort_values("rank").to_csv("data/2024_ski_rank.csv", index=False)
snbd_list.sort_values("rank").to_csv("data/2024_snbd_rank.csv", index=False)

# Appendix

In [ ]:
combined_df[((combined_df.racer_id == "McTaggart, Michael") | (combined_df.racer_id == "Parr, Jeff")) & (combined_df.year == 2014)]

In [12]:
combined_df[(combined_df.racer_id == "Daly, Thomas")]# & (combined_df.year == 2014)]

,race_id,discipline,racer_id,_time,year,first_name,last_name,gender,birth_year,spouse_id
1647,1017,SKI,"Daly, Thomas",27.58,2016,Thomas,Daly,-,NaN,NaN
1721,1018,SKI,"Daly, Thomas",26.79,2017,Thomas,Daly,-,NaN,NaN
1826,1019,SKI,"Daly, Thomas",27.27,2017,Thomas,Daly,-,NaN,NaN
1945,1020,SKI,"Daly, Thomas",41.30,2017,Thomas,Daly,-,NaN,NaN
2021,1021,SKI,"Daly, Thomas",45.50,2017,Thomas,Daly,-,NaN,NaN
2128,1022,SKI,"Daly, Thomas",35.90,2017,Thomas,Daly,-,NaN,NaN
2225,1023,SKI,"Daly, Thomas",26.69,2017,Thomas,Daly,-,NaN,NaN
2308,1024,SKI,"Daly, Thomas",27.27,2018,Thomas,Daly,-,NaN,NaN
2414,1025,SKI,"Daly, Thomas",24.85,2018,Thomas,Daly,-,NaN,NaN
2513,1026,SKI,"Daly, Thomas",39.73,2018,Thomas,Daly,-,NaN,NaN


In [ ]:
ski_n_snbd_list = eligible_pop[eligible_pop.Program == 'SKISNBD'][['racer_id']].merge(ski_aggregated_df, how="left", on="racer_id")
ski_n_snbd_list = ski_n_snbd_list.rename(columns={"all_time_avg_percentile": "ski_all_time_avg_percentile",
"median_percentile_2020": "ski_median_percentile_2020",
"median_percentile_2022": "ski_median_percentile_2022",
"median_percentile_2023": "ski_median_percentile_2023",
"rank": "ski_rank",})
ski_n_snbd_list = ski_n_snbd_list.merge(snbd_aggregated_df, how="left", on="racer_id")
ski_n_snbd_list = ski_n_snbd_list.rename(columns={"all_time_avg_percentile": "snbd_all_time_avg_percentile",
"median_percentile_2020": "snbd_median_percentile_2020",
"median_percentile_2022": "snbd_median_percentile_2022",
"median_percentile_2023": "snbd_median_percentile_2023",
"rank": "snbd_rank",})
ski_n_snbd_list["avg_rank"] = (ski_n_snbd_list.ski_rank + ski_n_snbd_list.snbd_rank) / 2

In [ ]:
ski_n_snbd_list.sort_values("avg_rank").to_csv("data/2024_ski_n_snbd_rank.csv", index=False)

In [11]:
ski_aggregated_df.head()

,racer_id,all_time_avg_percentile,median_percentile_2020,median_percentile_2022,median_percentile_2023,rank
0,"Abbott, Brad",0.540623,0.678161,NaN,NaN,0.678161
1,"Abbott, Tori",0.720351,0.855217,NaN,NaN,0.855217
2,"Alexander Ash, Sue",0.577699,NaN,NaN,NaN,0.577699
3,"Allan, Christine",0.929750,NaN,NaN,NaN,0.929750
4,"Ancion, Audrey",0.797814,NaN,NaN,NaN,0.797814


In [29]:
combined_df[(combined_df.racer_id == "Bredur, Audun")]# & (combined_df.year == 2014)]

,race_id,discipline,racer_id,_time,year,first_name,last_name,gender,birth_year,spouse_id


In [21]:
combined_df[(combined_df.racer_id == "Robertson, Todd")]# & (combined_df.year == 2014)]

,race_id,discipline,racer_id,_time,year,first_name,last_name,gender,birth_year,spouse_id
27,1,SKI,"Leistner, Linda",41.94,2014,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
65,1,SNBD,"Leistner, Linda",61.14,2014,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
113,2,SKI,"Leistner, Linda",30.42,2013,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
155,2,SNBD,"Leistner, Linda",40.52,2013,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
193,3,SKI,"Leistner, Linda",42.20,2013,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
...,...,...,...,...,...,...,...,...,...,...
3812,2035,SNBD,"Leistner, Linda",64.30,2022,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
3845,2036,SKI,"Leistner, Linda",41.49,2022,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
3864,2036,SNBD,"Leistner, Linda",51.45,2022,Linda,Leistner,F,1960.0,"Lavoie, Daniel"
3901,2037,SKI,"Leistner, Linda",51.42,2022,Linda,Leistner,F,1960.0,"Lavoie, Daniel"


In [25]:
combined_df[(combined_df.racer_id == "McLoughlin, Kevin")].tail(20)# & (combined_df.year == 2014)]

,race_id,discipline,racer_id,_time,year,first_name,last_name,gender,birth_year,spouse_id
2268,1023,SKI,"McLoughlin, Kevin",33.06,2017,Kevin,McLoughlin,M,NaN,NaN
2367,1024,SKI,"McLoughlin, Kevin",34.16,2018,Kevin,McLoughlin,M,NaN,NaN
2471,1025,SKI,"McLoughlin, Kevin",32.70,2018,Kevin,McLoughlin,M,NaN,NaN
2562,1026,SKI,"McLoughlin, Kevin",49.88,2018,Kevin,McLoughlin,M,NaN,NaN
2754,1028,SKI,"McLoughlin, Kevin",54.16,2018,Kevin,McLoughlin,M,NaN,NaN
2942,1030,SKI,"McLoughlin, Kevin",34.51,2019,Kevin,McLoughlin,M,NaN,NaN
3040,1031,SKI,"McLoughlin, Kevin",50.09,2019,Kevin,McLoughlin,M,NaN,NaN
3340,2029,SKI,"McLoughlin, Kevin",32.41,2020,Kevin,McLoughlin,M,NaN,NaN
3430,2030,SKI,"McLoughlin, Kevin",42.88,2020,Kevin,McLoughlin,M,NaN,NaN
3530,2031,SKI,"McLoughlin, Kevin",47.71,2020,Kevin,McLoughlin,M,NaN,NaN
